# kospi200 일별시세 가져오기

#### iframe 페이지 크롤링의 이해
- iframe구조인지 파악(눈치채기) :
    - 페이지가 바뀌는데, url이 안바뀐다.
    - 페이지 구조상 안바뀌는게 맞는 것 같을 때
- html에서 iframe 태그 찾기 -> src 가져오기
- requests.get(iframe_src)

In [ ]:
# 날짜 :    date
# 체결가:  price
# 전일비:  diff
# 등락률:  rate
# 거래량:  volume
# 거래대금:   amount

## 1. 1페이지 날짜, 체결가만 가져와서 출력하기

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
url = 'https://finance.naver.com/sise/sise_index_day.naver?code=KPI200'
resp = requests.get(url)
resp

<Response [200]>

In [5]:
soup = BeautifulSoup(resp.content, 'lxml')
date_tags = soup.select('td.date')
price_tags = soup.selct('td.number_1')

dates = []

for date_tag, price_tag in dates:
    print(date.text)

## 2. 1페이지 전체 데이터 가져와서 아래 각각의 리스트에 저장하기

In [17]:
dates, diffs, prices, rates, volumes, amounts = [] ,[], [], [], [], [] # 변수초기화 안하면 계속 추가됨

In [13]:
url = 'https://finance.naver.com/sise/sise_index_day.naver?code=KOSPI&page=1'
resp = requests.get(url)
resp

soup = BeautifulSoup

In [14]:
soup = BeautifulSoup(resp.content, 'lxml')
soup.select('td.date')

[<td class="date">2025.05.13</td>,
 <td class="date">2025.05.12</td>,
 <td class="date">2025.05.09</td>,
 <td class="date">2025.05.08</td>,
 <td class="date">2025.05.07</td>,
 <td class="date">2025.05.02</td>]

In [9]:
date_tags = soup.select('td.date')
num1_tags = soup.select('td.number_1')
diff_tags = soup.select('td.rate_down')
# diff_tags = soup.select('td.rate_down span.tah.p11')

price_tags = num1_tags[::4]
rate_tags = num1_tags[1::4]
volume_tags = num1_tags[2::4]
amount_tags = num1_tags[3::4]

len(date_tags), len(diff_tags), len(price_tags), len(rate_tags), len(volume_tags), len(amount_tags)

(6, 6, 6, 6, 6, 6)

In [18]:
for i in range(len(date_tags)):
    dates.append(date_tags[i].text)
    diffs.append(float(diff_tags[i].text.strip()))
    prices.append(float(price_tags[i].text.replace(',','')))
    rates.append(float(rate_tags[i].text.strip().replace('%','')))
    volumes.append(int(volume_tags[i].text.replace(',','')))
    amounts.append(int(amount_tags[i].text.replace(',','')))

dates, diffs, prices, rates, volumes, amounts

(['2025.05.13',
  '2025.05.12',
  '2025.05.09',
  '2025.05.08',
  '2025.05.07',
  '2025.05.02'],
 [0.7, 30.06, 2.21, 5.68, 14.01, 3.18],
 [2608.03, 2607.33, 2577.27, 2579.48, 2573.8, 2559.79],
 [0.03, 1.17, -0.09, 0.22, 0.55, 0.12],
 [113994, 409749, 485550, 471636, 546755, 585492],
 [1871366, 8527540, 8034853, 10270131, 8935449, 8327449])

-1.14%
-0.24%
+0.03%
+1.16%
+0.14%
+1.91%


## 3. 전일비는 하락의 경우, 음수로 처리하여 1페이지 전체 정보 데이터프레임으로 저장하기

In [ ]:
diffs = []

In [ ]:
url = 'https://finance.naver.com/sise/sise_index_day.naver?code=KPI200'
resp = requests.get(url)

soup = BeautifulSoup(resp.content, 'lxml')

diff_tags = soup.select('td.rate_down span')

for diff_tag in diff_tags:

    #print(diff_tag['class']) #음수 확인 절차
    diff = float(diff_tag.text.strip())
    if 'nv01' in diff_tag['class']:
        diff = -diff
    diffs.append(diff)
print(diffs)

['0.57', '2.05', '1.09', '1.10', '0.52', 5.59, 0.02, 0.16, 2.52, 0.05, -0.32]


,날짜,체결가,전일비,등락률,거래량,거래대금
0,2024.11.05,340.30,-2.85,-0.96,19558,1416885
1,2024.11.04,343.60,6.11,1.81,99332,6618146
2,2024.11.01,337.49,-1.83,-0.54,104589,6352550
3,2024.10.31,339.32,-5.68,-1.65,147442,9235509
4,2024.10.30,345.00,-3.28,-0.94,128038,7459147
5,2024.10.29,348.28,1.01,0.29,134509,7875682


,체결가,전일비,등락률,거래량,거래대금
count,6.000000,6.000000,6.000000,6.000000,6.000000e+00
mean,342.331667,-1.086667,-0.331667,105578.000000,6.492986e+06
std,4.018728,4.141481,1.226335,45898.757922,2.689570e+06
min,337.490000,-5.680000,-1.650000,19558.000000,1.416885e+06
25%,339.565000,-3.172500,-0.955000,100646.250000,6.418949e+06
50%,341.950000,-2.340000,-0.740000,116313.500000,7.038646e+06
75%,344.650000,0.300000,0.082500,132891.250000,7.771548e+06
max,348.280000,6.110000,1.810000,147442.000000,9.235509e+06


## 4. 전체 페이지 전체 데이터 가져와서 csv파일로 저장하기
- 코드 시행 시점 무관하게 맨 끝페이지까지 크롤링하도록 자동화할 것.

In [19]:
# 맨끝페이지 번호 가져오는 코드
url = 'https://finance.naver.com//sise/sise_index_day.naver?code=KPI200&page=1'
resp = requests.get(url)

soup = BeautifulSoup(resp.content, 'lxml')
lst_tags = soup.select('td.pgRR a') #맨뒤 아이콘 link에서 가져옴
href = lst_tags[0]['href'] # pgRR의 0번째 a herf 소환
href

'/sise/sise_index_day.naver?code=KPI200&page=796'

In [26]:
href.split('page=')[-1] #맨끝페이지 확인 방법

'796'

In [27]:
lst_pn = int(href.split('page=')[-1]) #맨끝페이지 확인 방법
lst_pn

796

In [44]:
dates, diffs, prices, rates, volumes, amounts = [] ,[], [], [], [], []

for page in range(796, lst_pn+1):
    url =f'https://finance.naver.com/sise/sise_index_day.naver?code=KPI200&page={page}'
    resp = requests.get(url)
    # print(page,resp)
    soup = BeautifulSoup(resp.content, 'lxml')

    date_tags = soup.select('td.date')
    diff_tags = soup.select('td.rate_down')

    num1_tags = soup.select('td.number_1')

    price_tags = num1_tags[::4]
    rate_tags = num1_tags[1::4]
    volume_tags = num1_tags[2::4]
    amount_tags = num1_tags[3::4]

    for i in range(len(date_tags)):
        if date_tags[i].text.strip():
            dates.append(date_tags[i].text)
            diffs.append(diff_tags[i].text.strip())
            prices.append(price_tags[i].text)
            rates.append(rate_tags[i].text.strip()) ## \n \t 깨지는 이유??
            volumes.append(volume_tags[i].text)
            amounts.append(amount_tags[i].text)
        else:
            break

dates, diffs, prices, rates, volumes, amounts

(['2006.01.10',
  '2006.01.09',
  '2006.01.06',
  '2006.01.05',
  '2006.01.04',
  '2006.01.03'],
 ['1.32', '0.57', '2.05', '1.09', '1.10', '0.52'],
 ['178.98', '180.30', '180.87', '178.82', '179.91', '178.81'],
 ['-0.73%', '-0.32%', '+1.15%', '-0.61%', '+0.62%', '+0.29%'],
 ['182,811', '191,222', '206,313', '206,332', '188,841', '158,041'],
 ['4,490,698',
  '4,641,279',
  '4,609,858',
  '5,522,148',
  '5,360,812',
  '4,161,549'])

In [ ]:
# 마지막 페이지 정보 누락되어있을 경우 확인


['2006.01.04', '2006.01.03']

In [46]:
import pandas as pd

df_ksp = pd.DataFrame({'날짜' : dates, '체결가' : prices, '전일비' : diffs,
              '등락률' : rates, '거래량' : volumes, '거래대금' : amounts})
df_ksp

,날짜,체결가,전일비,등락률,거래량,거래대금
0,2006.01.10,178.98,1.32,-0.73%,"182,811","4,490,698"
1,2006.01.09,180.30,0.57,-0.32%,"191,222","4,641,279"
2,2006.01.06,180.87,2.05,+1.15%,"206,313","4,609,858"
3,2006.01.05,178.82,1.09,-0.61%,"206,332","5,522,148"
4,2006.01.04,179.91,1.10,+0.62%,"188,841","5,360,812"
5,2006.01.03,178.81,0.52,+0.29%,"158,041","4,161,549"
